<a href="https://colab.research.google.com/github/fakelaboratory/publish/blob/main/Boiling_water_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install moderngl
!pip install pyrr
!pip install moviepy
import moderngl as gl
import numpy as np
import random
import pyrr as pyrr
from PIL import Image
import moviepy.editor as mpy
from google.colab import files

ctx = gl.create_context(standalone=True, backend='egl')
print(ctx.info)

In [4]:
ctx.enable(gl.BLEND)
prog = ctx.program(
    vertex_shader="""
        #version 330
        in vec4 in_vert;
        void main() {
            gl_Position = in_vert;
        }
    """,
    fragment_shader="""
        #version 330
        uniform vec4 u_color;
        out vec4 f_color;
        void main() {
            f_color = u_color;
        }
    """,
)
vertices = np.asarray([
    [-0.5 , -0.5 , -0.5, 1.0] ,
    [ 0.5 , -0.5 , -0.5, 1.0] ,
    [ 0.5 ,  0.5 , -0.5, 1.0] ,
    [ 0.5 ,  0.5 , -0.5, 1.0] ,
    [-0.5 ,  0.5 , -0.5, 1.0] ,
    [-0.5 , -0.5 , -0.5, 1.0] ,

    [-0.5 , -0.5 ,  0.5, 1.0] ,
    [ 0.5 , -0.5 ,  0.5, 1.0] ,
    [ 0.5 ,  0.5 ,  0.5, 1.0] ,
    [ 0.5 ,  0.5 ,  0.5, 1.0] ,
    [-0.5 ,  0.5 ,  0.5, 1.0] ,
    [-0.5 , -0.5 ,  0.5, 1.0] ,

    [-0.5 ,  0.5 ,  0.5, 1.0] ,
    [-0.5 ,  0.5 , -0.5, 1.0] ,
    [-0.5 , -0.5 , -0.5, 1.0] ,
    [-0.5 , -0.5 , -0.5, 1.0] ,
    [-0.5 , -0.5 ,  0.5, 1.0] ,
    [-0.5 ,  0.5 ,  0.5, 1.0] ,

    [ 0.5 ,  0.5 ,  0.5, 1.0] ,
    [ 0.5 ,  0.5 , -0.5, 1.0] ,
    [ 0.5 , -0.5 , -0.5, 1.0] ,
    [ 0.5 , -0.5 , -0.5, 1.0] ,
    [ 0.5 , -0.5 ,  0.5, 1.0] ,
    [ 0.5 ,  0.5 ,  0.5, 1.0] ,

    [-0.5 , -0.5 , -0.5, 1.0] ,
    [ 0.5 , -0.5 , -0.5, 1.0] ,
    [ 0.5 , -0.5 ,  0.5, 1.0] ,
    [ 0.5 , -0.5 ,  0.5, 1.0] ,
    [-0.5 , -0.5 ,  0.5, 1.0] ,
    [-0.5 , -0.5 , -0.5, 1.0] ,

    [-0.5 ,  0.5 , -0.5, 1.0] ,
    [ 0.5 ,  0.5 , -0.5, 1.0] ,
    [ 0.5 ,  0.5 ,  0.5, 1.0] ,
    [ 0.5 ,  0.5 ,  0.5, 1.0] ,
    [-0.5 ,  0.5 ,  0.5, 1.0] ,
    [-0.5 ,  0.5 , -0.5, 1.0]
],dtype='f4')

def transform(v,type,args):
  if(type=="rotation"):
    mat = pyrr.matrix44.create_from_axis_rotation([args[0],args[1],args[2]],args[3])
  if(type=="scale"):
    mat = pyrr.matrix44.create_from_scale([args[0],args[1],args[2]])
  if(type=="translation"):
    mat = pyrr.matrix44.create_from_translation([args[0],args[1],args[2]])
  for i in range(len(v)):
    v[i]= pyrr.matrix44.apply_to_vector(mat, v[i])

In [5]:
rand = 18#less number, less transformations in frame

class Molecule:
  def __init__(self,pos,size):
    self.v = np.copy(vertices)
    self.pos  = pos
    self.size = size
    self.state = 0 # - water, 1 - gas
    self.group = 0
    self.acc = 0
    self.speed = 0

class Water:
  def __init__(self,size):
    self.group_transform = {}
    self.group_transport = []
    self.molecules = []
    self.edge = int(size**(1./3.)) #quantity of molecules is volume, cube root is edge
    self.side = 1/self.edge # size of molecule, because the whole cube of water is 1(OpenGL screen width = 2), devide 1 for edge of molecules
    #place them in cube shape
    for i in range(self.edge):
       for j in range(self.edge):
           for k in range(self.edge):
              #create molecule in with position in cube, x y z multiplyed by side with shift to the center of screen
              new_molecule = Molecule(np.array([j*self.side+self.side/2-0.5,i*self.side+self.side/2-0.5,k*self.side]),self.side)
              transform(new_molecule.v,"scale",[self.side,self.side,self.side])
              transform(new_molecule.v,"translation",new_molecule.pos)
              transform(new_molecule.v,"rotation",[1.0,1.0,0.0,0.25])
              self.molecules.append(new_molecule)
  def connect(self,a,b):
    #in general if both not in groups, create group
    #if first in group but second not then connect second
    #if in different groups, merge groups and delete second group
    #with adding element to group_transform arrays, with id's of groups
    if(self.molecules[a].state==1 and self.molecules[b].state==1):
      if(self.molecules[a].group == 0 and self.molecules[b].group == 0):
        self.molecules[b].group=random.randint(0,1000000000)
        self.molecules[a].group=self.molecules[b].group
        self.group_transform[str(self.molecules[a].group)]= np.array([])
        self.group_transform[str(self.molecules[a].group)]= np.append(self.group_transform[str(self.molecules[a].group)],[int(a),int(b)])
      elif(self.molecules[a].group == 0 and self.molecules[b].group != 0):
        self.molecules[a].group = self.molecules[b].group
        self.group_transform[str(self.molecules[a].group)]= np.append(self.group_transform[str(self.molecules[a].group)],int(a))
      elif(self.molecules[a].group != 0 and self.molecules[b].group == 0):
        self.molecules[b].group = self.molecules[a].group
        self.group_transform[str(self.molecules[b].group)]= np.append(self.group_transform[str(self.molecules[b].group)],int(b))
      elif(self.molecules[a].group != self.molecules[b].group):
        temp = self.molecules[b].group
        self.molecules[b].group = self.molecules[a].group
        for x in self.group_transform[str(temp)]:
           self.molecules[int(x)].group = self.molecules[b].group
        self.group_transform[str(self.molecules[a].group)]= np.append(self.group_transform[str(self.molecules[a].group)],self.group_transform[str(temp)])
        if (str(temp) in self.group_transform) and (len(self.group_transform)>1):
          del self.group_transform[str(temp)]
    #print(self.group_transform)
  def reorganize(self):
    if(len(self.molecules)>self.edge**2):
      for i in range(self.edge**2):
        #start transformations on the bottom level
        if(random.randint(1,rand)==1):
          self.molecules[i].state = 1
        #register connected to transformation groups
        if (i==0 or (i % self.edge != self.edge -1 and (i+1) % self.edge != 0)):
          self.connect(i,i+1) #if nearest in vertical but not not in new string
        if (i+self.edge<self.edge**2):
          self.connect(i,i+self.edge) #if nearest in horizontal
        if (i-self.edge**2>=0):
          self.connect(i,i-self.edge**2)
    #check and fill transforamation groups
    ro = 1000
    g  = 9.80665
    a0 = 3.112*10**-10#size of molecule
    for group_key in self.group_transform.copy():
      force = ro*(len(self.group_transform[str(group_key)])*(a0**3))
      mass  = len(self.group_transform[str(group_key)])*((1.6735575*10**-27)*18)
      if force >= mass:
        #create one big molecule with position of first in the group
        group_molecule = Molecule(self.molecules[int(self.group_transform[str(group_key)][0])].pos,self.side)
        group_molecule.state = 2
        group_molecule.acc = force/mass
        transform(group_molecule.v,"scale",[group_molecule.size,group_molecule.size,group_molecule.size])
        transform(group_molecule.v,"translation",group_molecule.pos)
        transform(group_molecule.v,"rotation",[1.0,1.0,0.0,0.25])
        #add to group_transport
        self.group_transport.append(group_molecule)
        #fill transform group by water and delete transform group
        for x in self.group_transform[str(group_key)]:
          self.molecules[int(x)].state = 0
          self.molecules[int(x)].group = 0
        #delete molucules from the the surface(same quantity as in the group)
        self.molecules = self.molecules[:-1*len(self.group_transform[str(group_key)])]
        del self.group_transform[str(group_key)]
    #move up group_transport
    print(self.group_transport)
    for group in self.group_transport:
      delta = group.speed*(1/15)+(1/2)*group.acc*((1/15)**2)
      group.speed += group.acc*(1/15)
      transform(group.v,"translation",[0.0,delta,0.0])


In [ ]:

water = Water(10000) #in 100 ml 3.35*10^24

def render_frame(time):
  water.reorganize()

  fbo = ctx.framebuffer(
    color_attachments=[ctx.texture((512, 512), 3)]
  )
  fbo.use()
  fbo.clear(1.0, 1.0, 1.0, 1.0)

  #draw heat
  render = np.copy(vertices)
  transform(render,"scale",[2.125,1.0,0.0])
  transform(render,"translation",[0.125,-1.0,-0.5])
  transform(render,"rotation",[1.0,1.0,0.0,0.25])
  vbo = ctx.buffer(render.reshape(render.size).tobytes())
  vao = ctx.vertex_array(prog, vbo, "in_vert")
  u_color = prog["u_color"]
  u_color.value = [1.0,0.0,0.0,1.0]
  vao.render()

  #draw water and gas
  for i in range(0,len(water.molecules)):
    vbo = ctx.buffer(water.molecules[i].v.tobytes())
    vao = ctx.vertex_array(prog, vbo, "in_vert")
    if (water.molecules[i].state==0): #water
      u_color = prog["u_color"]
      u_color.value = [0.0,0.0,1.0,0.025]
    if (water.molecules[i].state==1): #gas
      u_color = prog["u_color"]
      u_color.value = [1.0,1.0,1.0,0.5]
    vao.render()
  for i in range(0,len(water.group_transport)):
    vbo = ctx.buffer(water.group_transport[i].v.tobytes())
    vao = ctx.vertex_array(prog, vbo, "in_vert")
    u_color = prog["u_color"]
    u_color.value = [1.0,1.0,1.0,0.5]
    vao.render()

  return np.array(Image.frombytes(
    "RGB", fbo.size, fbo.color_attachments[0].read(),
    "raw", "RGB", 0, -1
  ))

clip = mpy.VideoClip(render_frame, duration=18)
clip.write_gif("anim.gif",fps=15)
#now in files anim.gif